# Minimal dummy example with zerobnl

We consider the following two nodes graph as a minimal dummy example of two nodes exchanging a variable and randomly add or substract a given value from it:

<img src="./figures/model.png" alt="Model" style="width: 500px;"/>

The above graph can be created using zerobnl as follow:

In [1]:
import os
from zerobnl import Simulator as Sim

/home/pablo/Projects/UseZerOBNL/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


You can safely ignore the following error (it will also be in the nodes logs):

`RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88`

-> [https://github.com/numpy/numpy/pull/432]

In [2]:
sim = Sim()

sim.edit.add_meta(name="BaseMeta", set_attrs=["a"], get_attrs=["b"])

sim.edit.add_model(
    name="BaseModel",
    meta="BaseMeta",
    wrapper="wrapper_base.py",
    # dockerfile="Dockerfile_base",
)

sim.edit.add_node(name="Base0", model="BaseModel", init_values={"c": 0.50})

sim.edit.add_node(name="Base1", model="BaseModel", init_values={"c": 0.25})

sim.edit.add_link(get_node="Base0", get_attr="b", set_node="Base1", set_attr="a")
sim.edit.add_link(get_node="Base1", get_attr="b", set_node="Base0", set_attr="a")

grp0 = sim.edit.create_group("GRP0", "Base0")
grp1 = sim.edit.create_group("GRP1", "Base1")

sim.edit.create_sequence(grp0, grp1)
sim.edit.create_steps([60] * 10)

INFO :: Meta-model BaseMeta created.
INFO :: Model BaseModel created.
INFO :: Node Base0 created.
INFO :: Node Base1 created.
INFO :: Link created Base0 -> Base1.
INFO :: Link created Base1 -> Base0.
INFO :: The group ('Base0',) have been created.
INFO :: The group ('Base1',) have been created.
INFO :: The sequence [['GRP0', ('Base0',)], ['GRP1', ('Base1',)]] have been created.
INFO :: 10 steps have been created.


This simulation is based on `wrapper_base.py`, you can use to it as template to create your own. 

If you do not provide a dockerfile to `add_model()`, the [default](https://github.com/IntegrCiTy/zerobnl/blob/master/Dockerfiles/Dockerfile) one will be used.

In [3]:
sim.run_simulation()

INFO :: Starting the simulation's deployment...
INFO :: Starting simulation...
INFO :: Simulation finished in 0 min and 7 sec


If you see `INFO :: Simulation finished in X min and Y sec` it means everything went well.
You can find logs of the nodes in the file `nodes.log`, it's a text file you can open it directly in Jupyter or in your favorite text editor.

At the begining of the file you will find a serie of:

`Step X/10 : DO SOMETHING
 ---> 29d2f3226daf`
 
It's the logs of the creation of the Docker image, based on the provided Dockerfile (here `Dockerfile_base`).

Then all the logs are structures in the same way:

`<node>    | <level> :: <message>`

* `node` refers to the concerned simulation node or orchestrator
* `level` can be `DEBUG`: used for development purpose, `INFO`: giving you info on the running process, `WARNING`: warning you on action to make or some weird behaviour, `ERROR`: something went wrong and `CRITICAL`: something went really wrong.
* `message` is the body of the log, it describes what's happening.

You can also find information on the ongoing simulation in the file `activity.log` (in the root folder for the main processus and on the temporary folder for each node)

In [4]:
sim.results.connect_to_results_db()
sim.results.list_of_available_results

INFO :: Connected to 172.17.01 Redis DB


,IN/OUT,Node,Attribute
0,IN,Base1,a
1,OUT,Base0,b
2,IN,Base0,a
3,OUT,Base1,b
4,X,Base0,y
5,X,Base1,y


The name to the stored results are build as `<type>||<node>||<attribute>`.

`type` can be:
* `IN` if it's an input attribute (to set - stored automatically)
* `OUT` if it's an output attribute (to get - stored automatically)
* `X` if it's an internal value (stored by the user, using the `save_attribute()` method in the wrapper)

Knowing this, you can create matching pattern using `*` in order to properly select results.

In [5]:
for key, value in sim.results.get_results_by_pattern("*Base0*").items():
    print(key)
    print(value)

INFO :: Matching results: Base0 - b
INFO :: Matching results: Base0 - a
INFO :: Matching results: Base0 - y


IN||Base0||a
2000-01-01 00:01:00    0.00
2000-01-01 00:02:00   -0.25
2000-01-01 00:03:00   -0.25
2000-01-01 00:04:00    0.00
2000-01-01 00:05:00    0.50
2000-01-01 00:06:00    1.00
2000-01-01 00:07:00    0.50
2000-01-01 00:08:00    1.25
2000-01-01 00:09:00    0.50
dtype: float64
OUT||Base0||b
2000-01-01 00:01:00    0.00
2000-01-01 00:02:00   -0.50
2000-01-01 00:03:00   -0.25
2000-01-01 00:04:00    0.25
2000-01-01 00:05:00    0.50
2000-01-01 00:06:00    1.00
2000-01-01 00:07:00    0.50
2000-01-01 00:08:00    1.00
2000-01-01 00:09:00    0.75
2000-01-01 00:10:00    0.00
dtype: float64
X||Base0||y
2000-01-01 00:01:00    0.0
2000-01-01 00:02:00   -1.0
2000-01-01 00:03:00    0.0
2000-01-01 00:04:00    1.0
2000-01-01 00:05:00    1.0
2000-01-01 00:06:00    1.0
2000-01-01 00:07:00   -1.0
2000-01-01 00:08:00    1.0
2000-01-01 00:09:00   -1.0
2000-01-01 00:10:00   -1.0
dtype: float64


In [6]:
for key, value in sim.results.get_results_by_pattern("IN*Base0*").items():
    print(key)
    print(value)

INFO :: Matching results: Base0 - a


IN||Base0||a
2000-01-01 00:01:00    0.00
2000-01-01 00:02:00   -0.25
2000-01-01 00:03:00   -0.25
2000-01-01 00:04:00    0.00
2000-01-01 00:05:00    0.50
2000-01-01 00:06:00    1.00
2000-01-01 00:07:00    0.50
2000-01-01 00:08:00    1.25
2000-01-01 00:09:00    0.50
dtype: float64
